In [2]:
import tensorflow as tf
import numpy as np
from PIL import Image

In [4]:
def save_model_tflite(model, path):
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    tflite_model = converter.convert()
    with open(path, 'wb') as f:
        f.write(tflite_model)

    return os.path.getsize(path)

In [7]:
mobilenet_model = tf.keras.applications.MobileNetV2(weights="imagenet")
origin_size = save_model_tflite(mobilenet_model, "mobilenet.tflite")
print(f'origin_size : {origin_size}')

INFO:tensorflow:Assets written to: C:\Users\lswsj\AppData\Local\Temp\tmpzhz2qzye\assets
INFO:tensorflow:Assets written to: C:\Users\lswsj\AppData\Local\Temp\tmpzhz2qzye\assets
origin_size : 13988520


In [2]:
def save_model_tflite_dynamic_range_optimization(model, path):
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    tflite_model = converter.convert()
    with open(path, 'wb') as f:
        f.write(tflite_model)

    return os.path.getsize(path)

In [8]:
mobilenet_model = tf.keras.applications.MobileNetV2(weights="imagenet")
dr_size = save_model_tflite_dynamic_range_optimization(mobilenet_model,
    "mobilenet_dynamic_range.tflite")
print(f'dr_size : {dr_size}')
# 


INFO:tensorflow:Assets written to: C:\Users\lswsj\AppData\Local\Temp\tmpp4qtrghd\assets
INFO:tensorflow:Assets written to: C:\Users\lswsj\AppData\Local\Temp\tmpp4qtrghd\assets
dr_size : 3593600


In [11]:
def save_model_tflite_float16_optimization(model, path):
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    converter.target_spec.supported_types = [tf.float16]
    tflite_model = converter.convert()
    with open(path, 'wb') as f:
        f.write(tflite_model)

    return os.path.getsize(path)

In [13]:
mobilenet_model = tf.keras.applications.MobileNetV2(weights="imagenet")
fl16_size = save_model_tflite_float16_optimization(mobilenet_model, "mobilenet_float16.tflite")
print(f'fl16_size : {fl16_size}')


INFO:tensorflow:Assets written to: C:\Users\lswsj\AppData\Local\Temp\tmpf4kvcu6e\assets
INFO:tensorflow:Assets written to: C:\Users\lswsj\AppData\Local\Temp\tmpf4kvcu6e\assets
fl16_size : 7031872


In [124]:
def save_model_tflite_int_optimization(model, path, representative_dataset):
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    converter.representative_dataset = representative_dataset
    tflite_model = converter.convert()
    with open(path, 'wb') as f:
        f.write(tflite_model)

    return os.path.getsize(path)

In [4]:
def get_preprocessed_test_image(image_dir, count=100):
    files = os.listdir(image_dir)
    resized_images = np.array(np.zeros((count, 224, 224, 3)))
    for i in range(count):
        file = files[i]
        path = os.path.join(image_dir, file)
        image = np.array(Image.open(path))

        if len(np.shape(image)) == 2 :
            image = convert_channel(image)
        
        resized_images[i] = tf.image.resize(image, [224, 224])

    return resized_images


def convert_channel(img):
    return np.repeat(img[:, :, np.newaxis], 3, axis=2)

In [5]:
image_count = 100
image_data = get_preprocessed_test_image("./ILSVRC2012_img_val/", image_count)
image_data = np.array(tf.keras.applications.mobilenet.preprocess_input(image_data), np.float32)

def representative_dataset():
    for input_value in tf.data.Dataset.from_tensor_slices(image_data).batch(1).take(image_count):
        yield [input_value]

In [125]:
mobilenet_model = tf.keras.applications.MobileNetV2(weights="imagenet")
int_size = save_model_tflite_int_optimization(mobilenet_model, "mobilenet_int.tflite", representative_dataset)
print(f'int_size : {int_size}')

INFO:tensorflow:Assets written to: C:\Users\lswsj\AppData\Local\Temp\tmpq4__e242\assets
INFO:tensorflow:Assets written to: C:\Users\lswsj\AppData\Local\Temp\tmpq4__e242\assets
int_size : 4134304


In [136]:
def save_model_tflite_fullint_optimization(model, path, representative_dataset):
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    converter.representative_dataset = representative_dataset
    converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
    converter.inference_input_type = tf.uint8
    converter.inference_output_type = tf.uint8
    tflite_model = converter.convert()
    with open(path, 'wb') as f:
        f.write(tflite_model)

    return os.path.getsize(path)



In [137]:
mobilenet_model = tf.keras.applications.MobileNetV2(weights="imagenet")
fullint_size = save_model_tflite_fullint_optimization(mobilenet_model, "mobilenet_fullint.tflite", representative_dataset)
print(f'fullint_size : {fullint_size}')

INFO:tensorflow:Assets written to: C:\Users\lswsj\AppData\Local\Temp\tmp1mubvj3k\assets
INFO:tensorflow:Assets written to: C:\Users\lswsj\AppData\Local\Temp\tmp1mubvj3k\assets
fullint_size : 4134384


In [8]:
def run_mobilent_tflite_model(path, test_image_count=100):
    interpreter = tf.lite.Interpreter(str(path))
    interpreter.allocate_tensors()

    input_details = interpreter.get_input_details()[0]
    output_details = interpreter.get_output_details()[0]

    input_data = get_preprocessed_test_image("./ILSVRC2012_img_val/", test_image_count)

    if input_details['dtype'] == np.float32:
        input_data = tf.keras.applications.mobilenet.preprocess_input(input_data)
        input_data = np.array(input_data, np.float32)

    y_pred = np.array(np.zeros((len(input_data), 1000)), np.float32)
    for i in range(len(input_data)):
        if input_details['dtype'] == np.uint8:
            test_image = [np.array(input_data[i].astype(int), dtype=np.uint8)]
        else:
            test_image = [input_data[i]]
        
        interpreter.set_tensor(input_details['index'], test_image)
        interpreter.invoke()
        output = interpreter.get_tensor(output_details['index'])

        if output_details['dtype'] == np.uint8:
            output_param = output_details["quantization_parameters"]
            output_scale = output_param["scales"]
            output_zero_point = output_param["zero_points"]
            output = output.astype(np.float32)
            output = (output - output_zero_point) * output_scale

        y_pred[i] = output
    
    topK = 3
    y_pred_top = tf.keras.applications.mobilenet.decode_predictions(y_pred, top=topK)

    label_file = open("labels.txt", "r")
    label = label_file.read().split("\n")[:test_image_count]
    
    return check_accuracy(y_pred_top, label)


def check_accuracy(y_pred_top, label):
    success_count = 0

    for i in range(len(y_pred_top)):
        output_list = []

        for output in y_pred_top[i]:
            if isinstance(output, int):
                output_list.append(output)
            else:
                output_list.append(output[1])

        if label[i] in output_list:
            success_count += 1
        
    return success_count / len(y_pred_top)


In [9]:
origin_acc = run_mobilent_tflite_model("mobilenet.tflite")
dynamic_range_acc = run_mobilent_tflite_model("mobilenet_dynamic_range.tflite")
float16_acc = run_mobilent_tflite_model("mobilenet_float16.tflite")
int_acc = run_mobilent_tflite_model("mobilenet_int.tflite")
fullint_acc = run_mobilent_tflite_model("mobilenet_fullint.tflite")

f'{origin_acc}, {dynamic_range_acc}, {float16_acc}, {int_acc}, {fullint_acc}'

time : 0.0653083324432373
time : 0.14661645889282227
time : 0.07132840156555176
time : 2.0459702014923096
time : 2.054039716720581


'1.0, 0.0, 1.0, 0.0, 0.0'